# Import necessary libraries


In [ ]:
import pandas as pd 
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import json
import calendar
from collections import Counter
from plotly.offline import iplot
from plotly.subplots import make_subplots

# Read data from a CSV file into a pandas DataFrame and handle defects


In [ ]:
df = pd.read_csv('netflix_titles.csv')
df.info()

Check and display null rates for each column in the DataFrame

In [ ]:
for i in df.columns:
    null_rate = df[i].isna().sum() / len(df) * 100 
    if null_rate > 0 :
        print(f"{i} null rate: {round(null_rate, 2)}%")


Dealing with missing data
* Replace NaN values in specific columns with 'No Data'
* Drop rows with NaN values in any column
* Drop duplicate rows from the DataFrame

In [ ]:
df['country'].replace(np.nan, 'No Data', inplace  = True)
df['cast'].replace(np.nan, 'No Data', inplace  = True)
df['director'].replace(np.nan, 'No Data', inplace  = True)

df.dropna(inplace=True)

df.drop_duplicates(inplace= True)

Dealing with dates, seasons and durations

In [ ]:
df["date_added"] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors='coerce')
df['year_added'] = df['date_added'].dt.year.fillna(0).astype(int)
df['month_added'] = df['date_added'].dt.month.fillna(0).astype(int)

df['season_count'] = df.apply(lambda x: x['duration'].split(" ")[0] if "Season" in x['duration'] else "", axis=1)
df['duration'] = df.apply(lambda x: x['duration'].split(" ")[0] if "Season" not in x['duration'] else "", axis=1)
df.head()

In [ ]:
grouped_data = df.groupby('type').size().reset_index(name='count')

fig = px.pie(grouped_data, names='type', values='count', title='Distribution of Netflix Content Types',
             hole=0.3, color_discrete_sequence=px.colors.qualitative.Set3)


fig.update_layout(margin=dict(l=20, r=20, t=60, b=20), showlegend=True)

fig.show()

# Distribution of Netflix content types
This code generates a pie chart using Plotly to illustrate the distribution of Netflix content types. It begins by grouping the DataFrame by 'type' and calculating the count for each type. The pie chart is then created with specific settings such as title, colors, and layout adjustments. Finally, the interactive plot is displayed.

In [ ]:
col = "year_added"

df_filtered = df[df['year_added'] > 0]

d1 = df_filtered[df_filtered["type"] == "TV Show"]
d2 = df_filtered[df_filtered["type"] == "Movie"]

vc1 = d1[col].value_counts().reset_index()
vc1['percent'] = vc1['count'].apply(lambda x : 100 * x / sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2['percent'] = vc2['count'].apply(lambda x : 100 * x / sum(vc2['count']))
vc2 = vc2.sort_values(col)


Trend of content added over the years for TV Shows and Movies.

In [ ]:
trace1 = go.Scatter(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Scatter(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))

layout = go.Layout(title="Content added over the years",
                   xaxis=dict(title='Year'), yaxis=dict(title='Content'), 
                   legend=dict(x=0.1, y=1.1, orientation="h"))

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

Percentage of TV shows and movies added over the years. 

In [ ]:
trace3 = go.Scatter(x=vc1[col], y=vc1["percent"], name="TV Shows", line=dict(color="#a678de"))
trace4 = go.Scatter(x=vc2[col], y=vc2["percent"], name="Movies", line=dict(color="#6ad49b"))

layout = go.Layout(title="Percentage of Content added over the years",
                   xaxis=dict(title='Year'), yaxis=dict(title='Percentage'),
                   legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data=[trace3, trace4], layout=layout)
fig.show()

This code determines the year in which the maximum number of TV shows and movies were added, respectively.

In [ ]:
max_tv_shows_year = vc1.loc[vc1['count'].idxmax()][col]
max_movies_year = vc2.loc[vc2['count'].idxmax()][col]

print(f"The maximum number of TV Shows was added in {int(max_tv_shows_year)}.")
print(f"The maximum number of Movies was added in {int(max_movies_year)}.")


Distribution of release years for added TV shows and movies, with counts represented by different bars for each content type.

In [ ]:
col = "release_year"

df_filtered = df[df['release_year'] > 0]

d1 = df_filtered[df_filtered["type"] == "TV Show"]
d2 = df_filtered[df_filtered["type"] == "Movie"]

vc1 = d1[col].value_counts().reset_index()
vc1['percent'] = vc1['count'].apply(lambda x : 100 * x / sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2['percent'] = vc2['count'].apply(lambda x : 100 * x / sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Release year of added contents", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Titles and release years of the first 15 entries.

In [ ]:
small = df.sort_values("release_year", ascending = True)
small = small[small['duration'] != ""]
small[['title', "release_year"]][:15]

Titles and release years of the first 15 entries for TV shows

In [ ]:
small = df.sort_values("release_year", ascending = True)
small = small[small['season_count'] != ""]
small[['title', "release_year"]][:15]

Displays the distribution of added contents per month

In [ ]:
col = 'month_added'
df_filtered = df[df[col] > 0]

d1 = df_filtered[df_filtered["type"] == "TV Show"]
d2 = df_filtered[df_filtered["type"] == "Movie"]

vc1 = d1[col].value_counts().reset_index()
vc1['percent'] = vc1['count'].apply(lambda x : 100 * x / sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2['percent'] = vc2['count'].apply(lambda x : 100 * x / sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))

data = [trace1, trace2]
layout = go.Layout(title="In which month, the conent is added the most?", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)

fig.show()

Print the month in which the most TV shows and movies were added.

In [ ]:
max_tv_shows_year = vc1.loc[vc1['count'].idxmax()][col]
max_movies_year = vc2.loc[vc2['count'].idxmax()][col]

print(f"The most of the TV Shows was added in {calendar.month_name[int(max_tv_shows_year)]}.")
print(f"The most of the Movies was added in {calendar.month_name[int(max_movies_year)]}.")

This code generates a choropleth map and a horizontal bar chart to visualize the distribution of content across different countries.

In [ ]:
file_path = 'countries.json'

with open(file_path, 'r') as file:
    country_codes = json.load(file)

Distribution of movie durations

In [ ]:
x1 = d2['duration'].fillna(0.0).astype(float)
fig = ff.create_distplot([x1], ['a'], bin_size=0.7, curve_type='normal', colors=["#6ad49b"])
fig.update_layout(title_text='Distplot with Normal Distribution')
fig.show()

Distribution of the number of seasons for TV shows.

In [ ]:
col = 'season_count'
vc1 = d1[col].value_counts().reset_index()
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
data = [trace1]
layout = go.Layout(title="Seasons", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Compare the distribution of ratings for TV shows and movies

In [ ]:
col = "rating"

vc1 = d1[col].value_counts().reset_index()
vc1['percent'] = vc1['count'].apply(lambda x : 100*x/sum(vc1['count']))
vc1 = vc1.sort_values(col)

vc2 = d2[col].value_counts().reset_index()
vc2['percent'] = vc2['count'].apply(lambda x : 100*x/sum(vc2['count']))
vc2 = vc2.sort_values(col)

trace1 = go.Bar(x=vc1[col], y=vc1["count"], name="TV Shows", marker=dict(color="#a678de"))
trace2 = go.Bar(x=vc2[col], y=vc2["count"], name="Movies", marker=dict(color="#6ad49b"))
data = [trace1, trace2]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Visualize the distribution of content categories (genres) for movies

In [ ]:
col = "listed_in"
categories = ", ".join(d2['listed_in']).split(", ")
counter_list = Counter(categories).most_common(50)
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="#a678de"))

data = [trace1]
layout = go.Layout(title="Content added over the years", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Most common cast members in movies or TV shows from the United States, India, the United Kingdom, Canada, Spain, and Japan

In [ ]:
def country_trace(country, flag = "movie"):
    df_filtered = df[df["cast"] != "No Data"]
    df_filtered["from_us"] = df_filtered['country'].fillna("").apply(lambda x : 1 if country.lower() in x.lower() else 0)
    small = df_filtered[df_filtered["from_us"] == 1]
    if flag == "movie":
        small = small[small["duration"] != ""]
    else:
        small = small[small["season_count"] != ""]
    cast = ", ".join(small['cast'].fillna("")).split(", ")
    tags = Counter(cast).most_common(25)
    tags = [_ for _ in tags if "" != _[0]]

    labels, values = [_[0]+"  " for _ in tags], [_[1] for _ in tags]
    trace = go.Bar(y=labels[::-1], x=values[::-1], orientation="h", name="", marker=dict(color="#a678de"))
    return trace

traces = []
titles = ["United States", "", "India", "", "United Kingdom", "Canada", "", "Spain", "", "Japan"]
for title in titles:
    if title != "":
        traces.append(country_trace(title))

fig = make_subplots(rows=2, cols=5, subplot_titles=titles)
fig.add_trace(traces[0], 1,1)
fig.add_trace(traces[1], 1,3)
fig.add_trace(traces[2], 1,5)
fig.add_trace(traces[3], 2,1)
fig.add_trace(traces[4], 2,3)
fig.add_trace(traces[5], 2,5)

fig.update_layout(height=1200, showlegend=False)
fig.show()

Most common cast members in TV shows from the United States, the United Kingdom

In [ ]:
traces = []
titles = ["United States","", "United Kingdom"]
for title in titles:
    if title != "":
        traces.append(country_trace(title, flag="tv_shows"))

fig = make_subplots(rows=1, cols=3, subplot_titles=titles)
fig.add_trace(traces[0], 1,1)
fig.add_trace(traces[1], 1,3)

fig.update_layout(height=600, showlegend=False)
fig.show()

Display the most prolific movie directors from the United States based on the amount of content they have produced

In [ ]:
col = "director"
df_filtered = df[df[col] != "No Data"]
small = df_filtered[df_filtered["type"] == "Movie"]
small = small[small["country"] == "United States"]

categories = ", ".join(small[col].fillna("")).split(", ")
counter_list = Counter(categories).most_common(12)
counter_list = [_ for _ in counter_list if _[0] != ""]
labels = [_[0] for _ in counter_list][::-1]
values = [_[1] for _ in counter_list][::-1]
trace1 = go.Bar(y=labels, x=values, orientation="h", name="TV Shows", marker=dict(color="orange"))

data = [trace1]
layout = go.Layout(title="Movie Directors from US with most content", legend=dict(x=0.1, y=1.1, orientation="h"))
fig = go.Figure(data, layout=layout)
fig.show()

Contents of the movie directior with the most content.

In [ ]:
tag = "jay karas"
df["relevant"] = df['director'].fillna("").apply(lambda x : 1 if tag in x.lower() else 0)
small = df[df["relevant"] == 1]
small[['title', 'release_year', 'listed_in']]

In [ ]:
tag = "Stand-Up Comedy"
df["relevant"] = df['listed_in'].fillna("").apply(lambda x : 1 if tag.lower() in x.lower() else 0)
small = df[df["relevant"] == 1]
small[small["country"] == "United States"][["title", "country","release_year"]].head(10)